# Simulating a dynamic recommendation setting

In [1]:
import numpy as np
import pandas as pd
import sys
import pickle

import torch
from torch.utils.data import DataLoader, random_split

from tqdm import tqdm

sys.path.append('..')

from simulationConstants import ML_1M_FILLED_PATH, ML_1M_FILLED_PATH_PKL, ML_1M_ORACLE_PATH

In [2]:
synthetic_data_matrix = pd.read_csv(f"../{ML_1M_FILLED_PATH}")

In [3]:
synthetic_data_matrix

,user,item,clicked_and_examined,clicked_at,timestamp
0,1,1825,0.0,NaN,NaN
1,1,3666,0.0,NaN,NaN
2,1,1499,0.0,NaN,NaN
3,1,3876,0.0,NaN,NaN
4,1,12,NaN,NaN,NaN
...,...,...,...,...,...
1057795,6040,1286,NaN,NaN,NaN
1057796,6040,2163,NaN,NaN,NaN
1057797,6040,2118,NaN,NaN,NaN
1057798,6040,2658,0.0,NaN,NaN


In [4]:
len(synthetic_data_matrix["user"].drop_duplicates())

5289

In [5]:
feedback = synthetic_data_matrix.rename(columns={"clicked_and_examined": "relevant", "clicked_at": "click"})

In [6]:
initial_date = feedback["timestamp"].max()

In [7]:
movielensOraclePreferenceMatrix = pd.read_csv(f"../{ML_1M_ORACLE_PATH}").drop(columns=["Unnamed: 0"])

In [8]:
movielensOraclePreferenceMatrix

,user,item,genres,rating
0,1,1193,drama,1
1,1,661,animation|children's|musical,0
2,1,914,musical|romance,0
3,1,3408,drama,1
4,1,2355,animation|children's|comedy,1
...,...,...,...,...
19574584,6040,2258,action,0
19574585,6040,2845,drama,0
19574586,6040,3607,comedy|drama|western,1
19574587,6040,690,romance,0


## Simulation process

To simulate users, we'll: 

1. Retain 1000 random users from our synbtethic_data_matrix
2. For each k iteration in the simulation:
    3. Recommend 20 items to the user
        a. For calibration: we recommend 100 but calibrate to 20. This is done to increase recall before running the reranking
    4. Log the clicks 
5. Every 50 iteractions, retrain the recommender
6. Run this for 10_000 iteractions

In [9]:
from simulationUtils import get_user_feedback_for_item

In [10]:
with open('userToExpDistribution.pkl', 'rb') as f:
    movielensUserToExpDistribution = pickle.load(f)

In [11]:
initial_date = None

In [12]:
unique_users = list(feedback["user"].drop_duplicates())

In [13]:
if initial_date is None:
    initial_date = pd.Timestamp.now().timestamp()
user_to_up_to_date_timestamp = pd.DataFrame({
    "user": unique_users, 
    "delta_from_start": 0.0
})
user_to_up_to_date_timestamp["timestamp_dist"] = user_to_up_to_date_timestamp["user"].map(movielensUserToExpDistribution)

In [14]:
def map_recommendation_to_feedback(user, rec_list, matrix, userToExpDistribution):
    results = []
    max_delta = 0
    for idx, item in enumerate(rec_list):
        user, item, feedback, clicked_at, delta  = get_user_feedback_for_item(user, item, idx+1, matrix, userToExpDistribution)
        if (delta is not None and delta > max_delta):
            max_delta = delta
        feedback = (user, item, feedback, clicked_at, delta)
        results.append(feedback)
    return results, max_delta

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [16]:
sys.path.append('/home/caio/dev/bprMf')

In [17]:
from bpr_mf import bprMFLClickDebiasingDataloader, bprMF, bpr_train_with_debiasing, bpr_loss_with_reg_with_debiased_click
from utils import generate_bpr_dataset_with_click_data

In [18]:
bpr_dataset = generate_bpr_dataset_with_click_data(feedback, num_negatives=5)

In [19]:
bpr_dataset

,user,pos_item,click_position,neg_item
0,3,3408,6,1268
1,3,3408,6,2998
2,3,3408,6,2705
3,3,3408,6,3683
4,3,3408,6,1018
...,...,...,...,...
603525,6040,2300,9,2685
603526,6040,2300,9,19
603527,6040,2300,9,3052
603528,6040,2300,9,2438


In [20]:
n_users = bpr_dataset["user"].max() + 1
n_items = bpr_dataset.pos_item.max() + 1


In [21]:
n_users, n_items

(6041, 3953)

In [22]:
def train(model, data, train_ratio=0.8, debug=False):

    bpr_dataset = generate_bpr_dataset_with_click_data(data, num_negatives=5)
    data_bpr = bprMFLClickDebiasingDataloader(bpr_dataset)


    train_len = int(train_ratio * len(data_bpr))
    test_len = len(data_bpr) - train_len


    train_data, test_data = random_split(data_bpr, [train_len, test_len])



    dataloader_bpr_train = DataLoader(train_data, batch_size=256, shuffle=True)
    dataloader_bpr_test = DataLoader(test_data, batch_size=256, shuffle=True)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


    _, _ = bpr_train_with_debiasing(
        train_data_loader=dataloader_bpr_train,
        test_data_loader=dataloader_bpr_test,
        model=model,
        bpr_loss=bpr_loss_with_reg_with_debiased_click,
        optimizer=optimizer,
        reg_lambda=5e-4,
        debug=debug
    )

    return model

In [23]:
sys.path.append("/home/caio/dev/calibratedRecs")
sys.path.append("/home/caio/dev/calibratedRecs/constants.py")


In [24]:
from calibrationUtils import preprocess_genres

In [25]:

from constants import ITEM_COL, GENRE_COL, USER_COL

In [26]:
with open(f"../data/simulation/movielens_1m_sample_sinthetically_filled.pkl", "rb") as f:
    ml_filled = preprocess_genres(pickle.load(f).drop_duplicates())



In [27]:

ml_item_to_genre = (
    ml_filled[[ITEM_COL, GENRE_COL]]
    .set_index(ITEM_COL)[GENRE_COL]
    .to_dict()
)

In [28]:
from simulationUtils import simulate_user_feedback, get_candidate_items

In [29]:
from metrics import mace

In [30]:


def simulate(
    D, 
    model, 
    unique_users, 
    unique_items, 
    oracleMatrix, 
    userToExpDistribution, 
    item2genreMap, 
    k=100, 
    rounds=1000, 
    L=10, 
    initial_date=None
):
    """
    Simulates a dynamic recommendation setting.

    Parameters
    ----------
    D : pd.DataFrame
        Initial feedback data containing user-item interactions.
    model : torch.nn.Module
        Recommendation model with a predict_flat method.
    unique_users : list
        List of unique user IDs to simulate.
    unique_items : list
        List of unique item IDs to recommend.
    oracleMatrix : pd.DataFrame
        Matrix containing oracle user preferences for items.
    userToExpDistribution : dict
        Mapping from user ID to their timestamp distribution.
    item2genreMap : dict
        Mapping from item ID to its genres.
    k : int, optional
        Number of items to recommend per user per round (default=100).
    rounds : int, optional
        Number of simulation rounds (default=1000).
    L : int, optional
        Retrain model every L rounds (default=10).
    initial_date : float, optional
        Initial timestamp for simulation (default=None).

    Returns
    -------
    final_df : pd.DataFrame
        DataFrame containing all simulated feedback.
    maces : list
        List of MACE metric values computed every L rounds.
    """

    # Setup initial data
    unique_genres_in_items = set(g for genres in item2genreMap.values() for g in genres)
    n_genres = len(unique_genres_in_items)
    user2history = D.groupby(USER_COL).agg({ITEM_COL: list}).to_dict()[ITEM_COL]

    if initial_date is None:
        initial_date = pd.Timestamp.now().timestamp()
    user_to_up_to_date_timestamp = pd.DataFrame({
        "user": unique_users,
        "delta_from_start": 0.0
    })
    maces = []
    user_to_up_to_date_timestamp["timestamp_dist"] = user_to_up_to_date_timestamp["user"].map(userToExpDistribution)
    new_df = D.copy()
    for round in tqdm(range(1, rounds + 1), desc="Rounds"):
        rows_to_append = []
        for user in unique_users:
            candidate_items = torch.tensor(get_candidate_items(user, D, unique_items), device=device)
            row, user_to_up_to_date_timestamp = simulate_user_feedback(
                user=user,
                candidate_items=candidate_items,
                preference_matrix=oracleMatrix,
                recommend=model.predict_flat,
                k=k,
                user_to_up_to_date_timestamp=user_to_up_to_date_timestamp,
                userToExpDistribution=movielensUserToExpDistribution
            )
            rows_to_append.extend(row)
        recommendation_df = pd.DataFrame(rows_to_append, columns=new_df.columns)
        if (round % L == 0):
            print("retraining model...")
            model = train(model, new_df)
            print("Calculating mace")
            rec_df_grouped = recommendation_df.groupby(USER_COL).agg({ITEM_COL: list}).reset_index().rename(columns={ITEM_COL: "rec"})
            iteration_mace = mace(df=rec_df_grouped, user2history=user2history, recCol='rec', n_genres=n_genres, item2genreMap=item2genreMap)
            maces.append(iteration_mace)
        if (round % 100 == 0):
            recommendation_df.to_csv(f"data/movielens/no_calibration_sim_up_to_round_{round}")
        new_df = pd.concat([new_df, recommendation_df], ignore_index=True)
    final_df = pd.concat([D, new_df])
    final_df.loc[final_df["timestamp"].notnull(), "timestamp"] += initial_date
    return final_df, maces

In [31]:
unique_items = list(feedback["item"].unique())
unique_users = list(feedback["user"].unique())

In [32]:
model = bprMF(num_users=n_users, num_items=n_items, factors=30).to(device)
model = train(model, feedback)

In [ ]:
feedback_final, maces = simulate(
    D=feedback,
    model=model,
    unique_users=unique_users,
    unique_items=unique_items,
    oracleMatrix=movielensOraclePreferenceMatrix,
    userToExpDistribution=movielensUserToExpDistribution,
    k=100,
    rounds=1000,
    L=10,
    initial_date=0.0,
    item2genreMap=ml_item_to_genre
)

Rounds:   0%|          | 0/1000 [00:00<?, ?it/s]